In [ ]:
import pandas as pd
import numpy as np
from pymatgen.core.structure import Structure

In [ ]:
df = pd.read_csv('../data/xas_query.csv')
df['elements'] = df['elements'].apply(eval)
df['spectra'] = df['spectra'].apply(eval)
print('total spectra:', len(df))

In [ ]:
# remove entries without xas spectra
df = df[df['spectra'].str.len()>0].reset_index(drop=True)
print('total spectra:', len(df))

In [ ]:
# remove entries with NaNs in spectra
nan_ids = []
for i, entry in enumerate(df.itertuples()):
    for _, xas in entry.spectra.items():
        if np.any(np.isnan(xas['x'])) | np.any(np.isnan(xas['y'])):
            nan_ids += [i]
            break
        
df = df[~df.index.isin(nan_ids)].reset_index(drop=True)
print('total spectra:', len(df))

In [ ]:
# remove flat spectra
tol = 1e-3
flat_ids = []
for i, entry in enumerate(df.itertuples()):
    for _, xas in entry.spectra.items():
        if np.all(np.array(xas['y']) < tol):
            flat_ids += [i]
            break
        
df = df[~df.index.isin(flat_ids)].reset_index(drop=True)
print('total spectra:', len(df))

In [ ]:
# standardize column names
df = df.rename(columns={'sg': 'spacegroup'})

# standardize formula representation
df['structure'] = df['structure'].apply(eval).apply(Structure.from_dict)
df['structure'] = df['structure'].map(lambda x: x.get_primitive_structure())
df['formula'] = df['structure'].map(lambda x: x.formula)

In [ ]:
# save data
df['structure'] = df['structure'].map(lambda x: x.as_dict())
columns = ['icsd', 'formula', 'spacegroup', 'class', 'mp_id', 'structure', 'spectra']
df.to_csv('../data/data.csv', columns=columns, index=False)